##### Functions to build dataset out of labeled soundfiles, consisting of spectograms to train and validate learner 

In [35]:
%ls

 Volume in Laufwerk C: hat keine Bezeichnung.
 Volumeseriennummer: 46FB-03FE

 Verzeichnis von C:\Users\paulo\Coding\techlabsms-project-PredictivePollution

27.03.2021  17:43    <DIR>          .
27.03.2021  17:43    <DIR>          ..
06.09.2019  19:38             1.203 .gitignore
27.03.2021  16:58    <DIR>          .ipynb_checkpoints
27.03.2021  17:35         3.594.161 bg_lr_v2_5s_analyse.ipynb
27.03.2021  17:26            10.836 copy_engine_from_ESC-50.ipynb
27.03.2021  17:42            47.444 create_dataset.ipynb
27.03.2021  17:34             7.478 create_spectogram.ipynb
27.03.2021  17:41    <DIR>          datasets
27.03.2021  17:31             4.894 edit_soundfiles.ipynb
27.03.2021  17:26    <DIR>          ESC-50-master
27.03.2021  16:58           303.131 evaluation_learner_v2.ipynb
27.03.2021  16:58         1.268.986 learner_v1.ipynb
27.03.2021  16:58           724.533 learner_v2.ipynb
27.03.2021  16:58            10.078 merge_dataframes.ipynb
27.03.2021  17:41    <DIR>          p

In [34]:
from pathlib import Path
import pandas as pd
import random
import shutil

import pydub

import os
import imageio

import matplotlib
matplotlib.use('agg')

from matplotlib import pyplot as plt
from matplotlib import cm
from tqdm import tqdm
import pylab

import librosa
from librosa import display
import numpy as np

In [36]:
def create_spectrogram(source_filepath, destination_filepath):    
    y, sr = librosa.load(source_filepath, sr = 22050) # Use the default sampling rate of 22,050 Hz

    # Pre-emphasis filter
    pre_emphasis = 0.97
    y = np.append(y[0], y[1:] - pre_emphasis * y[:-1])

    # Compute spectrogram
    M = librosa.feature.melspectrogram(y, 
                                       sr, 
                                       fmax = sr/2, # Maximum frequency to be used on the on the MEL scale        
                                       n_fft=2048, 
                                       hop_length=512, 
                                       n_mels = 96, # As per the Google Large-scale audio CNN paper
                                       power = 2) # Power = 2 refers to squared amplitude
    # Power in DB
    log_power = librosa.power_to_db(M, ref=np.max)# Covert to dB (log) scale

    # Plotting the spectrogram and save as JPG without axes (just the image)
    pylab.figure(figsize=(5,5)) #was 14, 5
    pylab.axis('off') 
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    librosa.display.specshow(log_power, cmap=cm.jet)

    pylab.savefig(destination_filepath, bbox_inches=None, pad_inches=0)
    pylab.close()

In [37]:
def split_and_save(tgt_size, src_file, tgt_folder):
    
    
    if str(src_file)[-4:] == '.wav':
        soundf = pydub.AudioSegment.from_wav(src_file)
        fmt = 'wav'

    if str(src_file)[-4:] == '.mp3':
        soundf = pydub.AudioSegment.from_mp3(src_file)
        fmt = 'mp3'
    #soundf = pydub.AudioSegment.from_wav(src_file) #aktuell nur wav files -> mit if abfrage noch für mp3 anpassen
    for i, chunk in enumerate(soundf[::tgt_size]):
        #print(len(chunk))
        if len(chunk) == tgt_size: #soundfile wird nur gespeichert wenn es der target size entspricht --> letztes file wird verworfen wenn es zu kurz ist. Kann man sicher noch elleganter lösen in dem man letztes file mit leerer datei auffült oderso
            with open(tgt_folder + os.path.basename(src_file)[:-4] + "_%s.wav" % (i+1), "wb") as f:    
                chunk.export(f, format= 'wav')
                
    return()

In [38]:
def sflen(pth):
    if str(pth)[-4:] == '.wav':
        soundf = pydub.AudioSegment.from_wav(pth)
        sflen = len(soundf)
    elif str(pth)[-4:] == '.mp3':
        soundf = pydub.AudioSegment.from_mp3(pth)
        sflen = len(soundf)
    else:
        sflen = 0
        print("Warning: at least one file don't has valid datatype")
        
    return sflen

In [40]:
class Dataset():
    #class initialization
    def __init__(self, folderpath):
        self.pathlen = len(folderpath)
        self.folderpath = Path(folderpath)
        self.flst = list(self.folderpath.iterdir())
        self.lendict_p = {str(o)[:] : sflen(str(o)) for o in self.flst}
        self.lendict = {str(o)[len(folderpath):] : sflen(str(o)) for o in self.flst}
        self.lendf = pd.DataFrame({'soundfilelength' : self.lendict}) 
        
    #general information 
    def info(self):
        #return len(list(self.folderpath.iterdir()))
        print("Number of files: " + str(len(self.flst)))
        print('Overall length: '+ str(self.lendf['soundfilelength'].sum()) + " ms")
        print('Max file length: '+ str(self.lendf['soundfilelength'].max()) + " ms")
        print('Min file length: '+ str(self.lendf['soundfilelength'].min()) + " ms")
        print('mean file length: '+ str(self.lendf['soundfilelength'].mean()) + " ms")
    
    #return list of all files 
    def filelist(self):
        return self.flst
    
    #return length of soundfiles
    def oalen(self):
        oalen = self.lendf['soundfilelength'].sum()
        #for i in self.filelist():
         #   if str(i)[-3:] == 'wav':
          #      soundf = pydub.AudioSegment.from_wav(i)
           #     oalen = oalen + len(soundf)
            #if str(i)[-3:] == 'mp3':
             #   soundf = pydub.AudioSegment.from_mp3(i)
              #  oalen = oalen + len(soundf)
        return oalen
    
    #return longest soundfile
    def maxfile(self):
        maxfile = self.lendf['soundfilelength'].idxmax()
        return maxfile
    
    #return shortest soundfile
    def minfile(self):
        minfile = self.lendf['soundfilelength'].idxmin()
        return minfile
    
    #convert files to spectogram
    def convert_all(self, destination_filepath): #konvertiert alle files im ordner so wie sie sind zu spectrogrammen
        x = 1
        for i in self.flst:
            create_spectrogram(str(i), destination_filepath + str(i)[data1.pathlen:-4] + ".jpg")
            x = x +1
        print("converted " + str(x) + " files to spectrogram")
        
    #cut files into parts
    def split_all(self, tgt_size, tgt_folder):
        
        for i in self.flst:
            split_and_save(tgt_size, str(i), tgt_folder)
            
    # combine splitting and converting     
    def split_and_convert(self, tgt_size, tgt_folder, csv = False, label = '', ds_label = '' ): #evtl. noch optionale angaben ob man gesplitete soundfiles behalten möchte oder nicht implementieren
        if tgt_folder[-1] != '/':
            tgt_folder = tgt_folder + '/'
        
        
        if label == '':
            print('testprint:' + os.path.basename(tgt_folder[:-1]))
            label = os.path.basename(tgt_folder[:-1])

        
        if ds_label == '':
            ds_label = str(os.path.basename(tgt_folder[:-1]))
            
        
        
        tempfolder = 'temp/'
        #create folder for split files: (PATH).mkdir(exist_ok=True)
        if os.path.isdir(tempfolder) == False:
            os.makedirs(tempfolder)
            
        #create folder for spectograms
        if os.path.isdir(tgt_folder) == False:
            os.makedirs(tgt_folder)
            
        for i in self.flst:
            split_and_save(tgt_size, i, tempfolder)
            x = 1
            for j in list(Path(tempfolder).iterdir()):
                create_spectrogram(str(j), tgt_folder + os.path.basename(str(j))[:-4] + ".jpg")
                x = x +1
                os.remove(j)
            print("converted " + str(x) + " files to spectrogram")  
        if csv == True:    
            #create dataframe
            filelist = list(Path(tgt_folder).iterdir())

            labellist = [label] * len(filelist)
            ds_labellist = [ds_label] * len(filelist)
            pathlist = [os.path.relpath(tgt_folder)] * len(filelist)

            df = pd.DataFrame({'Filename' : filelist, 'Label' : labellist, 'DS_Label' : ds_labellist, 'Path' : pathlist})

            #clean filenames
            for i, row in df.iterrows():
                df.loc[i, 'Filename'] = str(os.path.basename(df.loc[i, 'Filename']))

            df.to_csv(str(os.path.dirname(tgt_folder) + '.csv'))
            print('Path to .csv: ' + str(os.path.dirname(tgt_folder) + '.csv'))
        
        #return df
        
    
    #files in dataset folder will be split in 3(learn, val und test) according to given formular (e.g. 0.7,0.2,0.1)
    def divide_data(self, tgt_dset_path, tgt_dist_learn = 0.7, tgt_dist_val = 0.2, tgt_dist_test = 0.1):
        
        if round(tgt_dist_learn + tgt_dist_val + tgt_dist_test, 4) != 1.0:
            print('sum of inputs are not 100%')
            return ()
        #calculate ideal distribution
        oalength = self.oalen()
        
        tgt_time_learn = tgt_dist_learn * oalength
        tmp_time_learn = 0
        tgt_time_val = tgt_dist_val * oalength
        tmp_time_val = 0
        tgt_time_test = tgt_dist_test * oalength
        tmp_time_test = 0
        
        
        #create folders
        if os.path.isdir(tgt_dset_path) == False:
            os.makedirs(tgt_dset_path)
        if os.path.isdir(tgt_dset_path + '/learn') == False:
            os.makedirs(tgt_dset_path + '/learn')
        if os.path.isdir(tgt_dset_path + '/val') == False:
            os.makedirs(tgt_dset_path + '/val')
        if os.path.isdir(tgt_dset_path + '/test') == False:
            os.makedirs(tgt_dset_path + '/test')
            
        learnpath = tgt_dset_path + '/learn'
        valpath = tgt_dset_path + '/val'
        testpath = tgt_dset_path + '/test'
        
        def dt_learn():
            return tmp_time_learn/(tgt_time_learn/100)
        
        def dt_val():
            return tmp_time_val/(tgt_time_val/100)
        
        def dt_test():
            return tmp_time_test/(tgt_time_test/100)
        
        #create random list
        randomlist = list(self.filelist())
        random.shuffle(randomlist)
        
        #go through random list and copy to folder with the biggest error/delta
        for i in randomlist:
            if (dt_learn() < 100) and (dt_learn() <= dt_val()) and (dt_learn() <= dt_test()):
                shutil.copy(i, learnpath)
                tmp_time_learn = tmp_time_learn + self.lendict_p[str(i)]
                
                
            elif (dt_val() < 100) and (dt_val() <= dt_learn()) and (dt_val() <= dt_test()):
                shutil.copy(i, valpath)
                tmp_time_val = tmp_time_val + self.lendict_p[str(i)]
                
            elif (dt_test() < 100) and (dt_test() <= dt_val()) and (dt_test() <= dt_learn()):
                shutil.copy(i, testpath)
                tmp_time_test = tmp_time_test + self.lendict_p[str(i)]
                
        print('error learn: ' + str(round(dt_learn() - 100, 2)) + '%; error val: ' + str(round(dt_val() - 100, 2)) + '%; error test: ' + str(round(dt_test() - 100, 2)) + '%;')

In [24]:
divide_path_benzin = 'C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/benzin_div'
divide_path_diesel = 'C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data_div/diesel_div'

In [26]:
benzin_train = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/benzin_div/learn')
benzin_valid = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/benzin_div/val')
benzin_test = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/benzin_div/test')

In [27]:
diesel_train = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/diesel_div/learn')
diesel_valid = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/diesel_div/val')
diesel_test = Dataset('C:/Users/paulo/Coding/techlabsms-project-PredictivePollution/pp_raw_data/pp_raw_data_div/diesel_div/test')

In [28]:
benzin_train.info()

Number of files: 0
Overall length: 0.0 ms
Max file length: nan ms
Min file length: nan ms
mean file length: nan ms


In [30]:
benzin_train.minfile()

ValueError: attempt to get argmin of an empty sequence

In [29]:
benzin_valid.info()

Number of files: 0
Overall length: 0.0 ms
Max file length: nan ms
Min file length: nan ms
mean file length: nan ms


In [13]:
benzin_valid.minfile()

'/dbsacc.mp3'

In [14]:
benzin_test.info()

Number of files: 47
Overall length: 2136916 ms
Max file length: 770972 ms
Min file length: 6991 ms
mean file length: 45466.29787234042 ms


In [15]:
diesel_train.info()

Number of files: 41
Overall length: 13727214 ms
Max file length: 3641842 ms
Min file length: 5000 ms
mean file length: 334810.0975609756 ms


In [16]:
diesel_valid.info()

Number of files: 7
Overall length: 4787585 ms
Max file length: 1711387 ms
Min file length: 5000 ms
mean file length: 683940.7142857143 ms


In [17]:
diesel_test.info()

Number of files: 11
Overall length: 2407575 ms
Max file length: 1334729 ms
Min file length: 5000 ms
mean file length: 218870.45454545456 ms


In [32]:
milliseconds = 1000
foldername = 'dataset1_1sec'

In [33]:
benzin_train.split_and_convert(milliseconds, ('datasets/' + foldername + '/train/benzin'), csv = True , label = 'benzin', ds_label = 'train')

Path to .csv: datasets/dataset1_1sec/train/benzin.csv


In [20]:
benzin_valid.split_and_convert(milliseconds, ('datasets/' + foldername + '/valid/benzin'), csv = True , label = 'benzin', ds_label = 'valid')

converted 11 files to spectrogram
converted 22 files to spectrogram
converted 22 files to spectrogram
converted 8 files to spectrogram
converted 8 files to spectrogram
converted 12 files to spectrogram
converted 12 files to spectrogram
converted 21 files to spectrogram
converted 14 files to spectrogram
converted 25 files to spectrogram
converted 16 files to spectrogram
converted 645 files to spectrogram
converted 614 files to spectrogram
converted 19 files to spectrogram
converted 18 files to spectrogram
converted 20 files to spectrogram
converted 504 files to spectrogram
converted 8 files to spectrogram
converted 11 files to spectrogram
converted 11 files to spectrogram
converted 13 files to spectrogram
converted 10 files to spectrogram
converted 14 files to spectrogram
converted 30 files to spectrogram
converted 11 files to spectrogram
converted 22 files to spectrogram
converted 21 files to spectrogram
converted 20 files to spectrogram
converted 11 files to spectrogram
converted 15 f

In [21]:
benzin_test.split_and_convert(milliseconds, ('datasets/' + foldername + '/test/benzin'), csv = True , label = 'benzin', ds_label = 'test')

converted 19 files to spectrogram
converted 8 files to spectrogram
converted 17 files to spectrogram
converted 706 files to spectrogram
converted 14 files to spectrogram
converted 11 files to spectrogram
converted 16 files to spectrogram
converted 7 files to spectrogram
converted 9 files to spectrogram
converted 11 files to spectrogram
converted 17 files to spectrogram
converted 14 files to spectrogram
converted 15 files to spectrogram
converted 38 files to spectrogram
converted 27 files to spectrogram
converted 19 files to spectrogram
converted 21 files to spectrogram
converted 10 files to spectrogram
converted 20 files to spectrogram
converted 13 files to spectrogram
converted 771 files to spectrogram
converted 12 files to spectrogram
converted 21 files to spectrogram
converted 17 files to spectrogram
converted 10 files to spectrogram
converted 16 files to spectrogram
converted 16 files to spectrogram
converted 8 files to spectrogram
converted 15 files to spectrogram
converted 11 fil

In [22]:
diesel_train.split_and_convert(milliseconds, ('datasets/' + foldername + '/train/diesel'), csv = True , label = 'diesel', ds_label = 'train')

converted 1389 files to spectrogram
converted 6 files to spectrogram
converted 179 files to spectrogram
converted 35 files to spectrogram
converted 610 files to spectrogram
converted 32 files to spectrogram
converted 6 files to spectrogram
converted 6 files to spectrogram
converted 1921 files to spectrogram
converted 504 files to spectrogram
converted 31 files to spectrogram
converted 6 files to spectrogram
converted 30 files to spectrogram
converted 6 files to spectrogram
converted 113 files to spectrogram
converted 6 files to spectrogram
converted 309 files to spectrogram
converted 6 files to spectrogram
converted 1280 files to spectrogram
converted 11 files to spectrogram
converted 198 files to spectrogram
converted 6 files to spectrogram
converted 290 files to spectrogram
converted 33 files to spectrogram
converted 431 files to spectrogram
converted 6 files to spectrogram
converted 3642 files to spectrogram
converted 6 files to spectrogram
converted 107 files to spectrogram
convert

In [23]:
diesel_valid.split_and_convert(milliseconds, ('datasets/' + foldername + '/valid/diesel'), csv = True , label = 'diesel', ds_label = 'valid')

converted 6 files to spectrogram
converted 876 files to spectrogram
converted 1483 files to spectrogram
converted 19 files to spectrogram
converted 1712 files to spectrogram
converted 179 files to spectrogram
converted 516 files to spectrogram
Path to .csv: datasets/dataset1_1sec/valid/diesel.csv


In [24]:
diesel_test.split_and_convert(milliseconds, ('datasets/' + foldername + '/test/diesel'), csv = True , label = 'diesel', ds_label = 'test')

converted 492 files to spectrogram
converted 6 files to spectrogram
converted 6 files to spectrogram
converted 41 files to spectrogram
converted 6 files to spectrogram
converted 1335 files to spectrogram
converted 204 files to spectrogram
converted 287 files to spectrogram
converted 6 files to spectrogram
converted 27 files to spectrogram
converted 6 files to spectrogram
Path to .csv: datasets/dataset1_1sec/test/diesel.csv
